In [ ]:
import pandas as pd
import numpy as np
import holidays
import matplotlib.pyplot as plt
import seaborn as sns

sns.set(style="whitegrid")

# --- Параметри ---
start_date = '2023-01-01'
end_date = '2024-12-31'
dates = pd.date_range(start=start_date, end=end_date)
n_days = len(dates)

# --- Категорії і товари ---
categories = {
    'Молочні продукти': [
        'Piątnica Mleko 3.2%', 'Mlekovita Ser żółty plasterki', 'President Masło extra',
        'Alpro Napój sojowy', 'Zott Jogurt naturalny'
    ],
    'Консерви': [
        'Łowicz Groszek konserwowy', 'Krakus Kukurydza konserwowa', 'Winogronki w occie',
        'Tarsmak Fasola czerwona', 'Cenos Tuńczyk kawałki w sosie własnym'
    ],
    'Іграшки': [
        'LEGO Classic 10717', 'Hasbro Monopoly', 'Playmobil City Life',
        'Smoby Traktor', 'Clementoni Puzzle 500'
    ],
    'Канцелярія': [
        'Stabilo Marker tekstowy', 'Pilot Długopis G2', 'Leitz Segregator A4',
        'Oxford Notes 100 kartek', 'Pentel Ołówek automatyczny'
    ],
    'Алкоголь': [
        'Żubrówka Bison Grass Vodka', 'Tyskie Piwo jasne', 'Cydr Lubelski',
        'Krakus Wino czerwone', 'Warka Radler cytryna'
    ],
    'Вода': [
        'Cisowianka Woda mineralna', 'Nałęczowianka Woda gazowana',
        'Żywiec Zdrój Naturalna', 'Evian Naturalna', 'Borjomi Woda mineralna'
    ],
    'Яйця': [
        'Jaja świeże klasy M', 'Jaja ekologiczne klasa L', 'Jaja wiejskie klasa S'
    ]
}

items = []
item_id = 1
for cat, names in categories.items():
    for name in names:
        items.append({'item_id': item_id, 'item_name': name, 'category': cat})
        item_id += 1

while len(items) < 650:
    for cat, names in categories.items():
        for name in names:
            if len(items) >= 650:
                break
            new_name = f"{name} Edycja {np.random.randint(1, 100)}"
            items.append({'item_id': item_id, 'item_name': new_name, 'category': cat})
            item_id += 1

items_df = pd.DataFrame(items[:650])

pl_holidays = holidays.Poland(years=[2023, 2024])

def sales_pattern_for_item(item_idx):
    base_level = {
        'Молочні продукти': 30,
        'Консерви': 20,
        'Іграшки': 10,
        'Канцелярія': 15,
        'Алкоголь': 12,
        'Вода': 25,
        'Яйця': 18
    }
    cat = items_df.loc[item_idx, 'category']
    base = base_level.get(cat, 10)
    
    trend = np.linspace(base, base*1.3, n_days)
    seasonality = 0.2 * base * np.sin(2 * np.pi * dates.dayofyear / 365.25 + item_idx)
    week_season = 0.15 * base * np.sin(2 * np.pi * dates.dayofweek / 7 + item_idx)
    noise = np.random.normal(0, base*0.3, n_days)
    
    sales = trend + seasonality + week_season + noise
    
    def holiday_adj(date):
        if date in pl_holidays:
            return 0.5
        for i in range(1,4):
            if (date + pd.Timedelta(days=i)) in pl_holidays:
                return 1 + 0.3 * (4 - i)
        return 1.0
    
    adjustments = np.array([holiday_adj(d) for d in dates])
    sales *= adjustments
    sales = np.clip(sales, 0, None)
    sales = sales.astype(int)
    return sales

all_data = []
for idx in range(len(items_df)):
    sales_qty = sales_pattern_for_item(idx)
    item_data = pd.DataFrame({
        'item_id': items_df.loc[idx, 'item_id'],
        'item_name': items_df.loc[idx, 'item_name'],
        'category': items_df.loc[idx, 'category'],
        'date': dates,
        'sales_qty': sales_qty
    })
    all_data.append(item_data)

df_sales = pd.concat(all_data, ignore_index=True)

# --- Аналітика ---

print("=== Інформація про датафрейм ===")
print(df_sales.info())

print("\n=== Описова статистика по продажах ===")
print(df_sales['sales_qty'].describe())

print("\n=== Унікальні товари ===", df_sales['item_id'].nunique())
print("=== Унікальні дати ===", df_sales['date'].nunique())

# --- Візуалізації ---

plt.figure(figsize=(12,6))
df_sales.groupby('date')['sales_qty'].sum().plot()
plt.title('Загальний щоденний обсяг продажів усіх товарів')
plt.xlabel('Дата')
plt.ylabel('Продажі (к-ть)')
plt.grid(True)
plt.show()

# Продажі по категоріях (середньоденні)
plt.figure(figsize=(12,6))
df_sales.groupby(['category','date'])['sales_qty'].sum().unstack(level=0).mean().plot(kind='bar')
plt.title('Середньоденні продажі по категоріях (2023-2024)')
plt.xlabel('Категорія')
plt.ylabel('Середні продажі')
plt.show()

# Топ-10 товарів за сумарними продажами за весь період
top_10 = df_sales.groupby('item_name')['sales_qty'].sum().sort_values(ascending=False).head(10)
plt.figure(figsize=(12,6))
sns.barplot(x=top_10.values, y=top_10.index)
plt.title('Топ-10 товарів за сумою продажів (2023-2024)')
plt.xlabel('Сумарні продажі')
plt.ylabel('Товар')
plt.show()

# Сезонність по місяцях для всього обсягу продажів
df_sales['month'] = df_sales['date'].dt.month
monthly_sales = df_sales.groupby('month')['sales_qty'].sum()
plt.figure(figsize=(12,6))
sns.lineplot(x=monthly_sales.index, y=monthly_sales.values, marker='o')
plt.title('Сезонність продажів по місяцях')
plt.xlabel('Місяць')
plt.ylabel('Сумарні продажі')
plt.xticks(range(1,13))
plt.grid(True)
plt.show()

# Аналіз по святам: середній продаж у святкові дні vs будні
df_sales['is_holiday'] = df_sales['date'].isin(pl_holidays)
holiday_sales = df_sales.groupby('is_holiday')['sales_qty'].mean()
plt.figure(figsize=(8,5))
sns.barplot(x=['Будні', 'Свята'], y=holiday_sales.values)
plt.title('Середній обсяг продажів: свята vs будні')
plt.ylabel('Середні продажі')
plt.show()

df_sales.to_csv('sales_data.csv', index=False)

